# 🛡️ Interactive Squad Simulation
Follow the directions at the top of each cell. After running the last cell, use the sliders and dropdowns below to run and visualize the simulation.

Colab Environment

In [1]:
# If you are working in colab, use this block to clone the GitHub repo, install dependencies, and pull down the latest changes.
%cd /content/
!rm -rf SE3250-Spring2025-SquadSimulation
!git clone https://github.com/SuprMunchkin/SE3250-Spring2025-SquadSimulation.git
%cd SE3250-Spring2025-SquadSimulation
%pip install -r requirements.txt

!git fetch origin
!git checkout "main" #Change this line to use your branch
%ls

/content
Cloning into 'SE3250-Spring2025-SquadSimulation'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 312 (delta 74), reused 53 (delta 42), pack-reused 206 (from 2)
Receiving objects: 100% (312/312), 3.57 MiB | 14.92 MiB/s, done.
Resolving deltas: 100% (137/137), done.
/content/SE3250-Spring2025-SquadSimulation
Already on 'main'
Your branch is up to date with 'origin/main'.
app.py                                          squad_simulator.ipynb
config/                                         tests/
DMV_Area_Group_2_Iteration_2_Code_Scraps.ipynb  test_simulation.sh*
models/                                         view/
requirements.txt


VS Code Environment

In [ ]:
%pip install -r ./requirements.txt

In [ ]:
# Import and setup
import sys
import yaml
import matplotlib.pyplot as plt
from ipywidgets import interact

# Custom imports
import os
sys.path.append("../models")
from models.squad_simulation import run_simulation
yaml_path = "config/simulation.yaml"
with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

map_size = config['map_size']

# Define interactive runner
def run_interactive_sim(blue_stock, red_stock, direction_deviation, armor_type, environment):
    """
    Run the interactive squad simulation and plot the results.

    Parameters:
        blue_stock (int): Number of blue units.
        red_stock (int): Number of red units.
        direction_deviation (int): Direction deviation in degrees.
        armor_type (str): Type of armor for blue units.
        environment (str): Simulation environment.
    """
    params = {
        "blue_stock": blue_stock,
        "red_stock": red_stock,
        "direction_deviation": direction_deviation,
        "armor_type": armor_type,
        "environment": environment
    }
    result = run_simulation(params, full_log=True)

    blue_positions = result['blue']['position_history']
    red_position = result['red']['current_position']

    plt.figure(figsize=(8, 8))

    # Plot Blue Patrol Path
    if blue_positions:  # Check if there are any blue positions
        x_vals, y_vals = zip(*blue_positions)
        plt.plot(x_vals, y_vals, label='Blue Patrol Path', color='blue')
        plt.scatter(x_vals[0], y_vals[0], c='green', label='Start', zorder=5)
        plt.scatter(x_vals[-1], y_vals[-1], c='purple', label='End', zorder=5)

    # Plot red Position(s)
    if red_position: # Check if red_positions exist
        # Check if it's a list of positions (moving) or a single position (stationary)
        if isinstance(red_position[0], (list, tuple)):
            # It's a list of positions, plot the path
            hx_vals, hy_vals = zip(*red_position)
            plt.plot(hx_vals, hy_vals, label='red Path', linestyle='--', color='red')
        else:
            # It's a single position, plot a scatter point
            hx, hy = red_position # Unpack the single coordinate pair
            plt.scatter(hx, hy, c='red', label='red Position', zorder=5)
    else:
        print("Warning: No red position data available.")

    plt.xlim(0, map_size)
    plt.ylim(0, map_size)
    plt.title("Squad Movement Simulation")
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

    print(f"👥 Blue Remaining: {result['blue']['stock']} / {params['blue_stock']}")
    print(f"🔴 red Remaining: {result['red']['stock']} / {params['red_stock']}")

    # Print statements here can be used for troublshooting.
    #print("Blue positions: ", positions)
    #print("red positions: ", red_positions)

# Create interactive widget interface
interact(
    run_interactive_sim,
    blue_stock=(1, 20),
    red_stock=(1, 40),
    direction_deviation=(0, 45, 5),
    armor_type=list(config['armor_profiles'].keys()),
    environment=list(config['threat_probs'].keys())
)

In [7]:
from ipywidgets import Button, VBox, Output
import itertools
import pandas as pd
import sys
import yaml
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display # Import display
from tqdm.notebook import tqdm # Import tqdm for notebooks
import pprint

# Custom imports
import os
sys.path.append("../models")
from models.squad_simulation import run_simulation
yaml_path = "config/simulation.yaml"
with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

pp = pprint.PrettyPrinter(indent=4)

# Define the number of simulations to run
number_of_runs = 100

# Prepare combinations of armor and environment
armor_types = list(config['armor_profiles'].keys())
environments = list(config['threat_probs'].keys())
combinations = list(itertools.product(armor_types, environments))

output = Output()

TBK = {}
THK = {}
TPD = {}

def plot_histograms(data_dict, title_prefix, xlabel, ylabel, step=1):
    fig, axes = plt.subplots(3, 3, figsize=(9, 9))
    axes = axes.flatten()
    fig.subplots_adjust(hspace=0.5, wspace=0.3)
    for i, (key, values) in enumerate(data_dict.items()):
        ax = axes[i]
        if values:
            bins = range(int(min(values)), int(max(values)) + 2)
            ax.hist(values, bins=bins, align='left', rwidth=0.4)
            ax.set_title(f'{title_prefix}\nfor {key}', fontsize=8)
            ax.set_xlabel(xlabel, fontsize=6)
            ax.set_ylabel(ylabel, fontsize=6)
            ax.grid(axis='y', alpha=0.75)
            min_val = int(min(values))
            max_val = int(max(values))
            ax.set_xticks(range(min_val, max_val + 1, step))
            # Add mean and standard deviation lines
            mean = np.mean(values)
            std = np.std(values)
            ax.axvline(mean, color='red', linestyle='--', label=f'Mean: {mean:.2f}')
            if len(values) > 1:  # Only plot std if enough data
              ax.axvline(mean + std, color='green', linestyle=':', label=f'Std: {std:.2f}')
              ax.axvline(mean - std, color='green', linestyle=':')
            # Add grid and legend
            ax.grid(True, axis='y', linestyle='--', alpha=0.3)
            ax.legend(fontsize=6)
            ax.legend()
        else:
            ax.set_title(f'{title_prefix}\nfor {key}', fontsize=8)
            ax.text(0.5, 0.5, 'No data to display', ha='center', va='center', transform=ax.transAxes)
    plt.show()

def plot_line(results, armor_types, y_key, title, ylabel):
    fig, ax = plt.subplots(figsize=(10, 6))
    for armor in armor_types:
        envs = [r['Environment'] for r in results if r['Armor'] == armor]
        ys = [r[y_key] for r in results if r['Armor'] == armor]
        ax.plot(envs, ys, marker='o', label=f'Armor: {armor}')
    ax.set_title(title)
    ax.set_xlabel("Environment")
    ax.set_ylabel(ylabel)
    ax.legend()
    plt.show()

def run_all_combinations(_):
    output.clear_output()
    blue_stock = 10
    red_stock = 40
    direction_deviation = 10

    results = []
    with output:
        # Wrap the outer loop with tqdm to show progress.
        for armor, env in tqdm(combinations, desc="Processing Combinations"):
            total_blue_remaining = 0
            total_red_remaining = 0
            total_red_spawned = 0
            total_effective_movement = 0
            total_blue_kills = 0
            total_red_kills = 0
            tbk_list = []
            TBK[f"{armor}\n+{env}"] = tbk_list
            thk_list = []
            THK[f"{armor}\n+{env}"] = thk_list
            tpd_list = []
            TPD[f"{armor}\n+{env}"] = tpd_list

            # Wrap the inner loop with tqdm to show progress.
            for _ in tqdm(range(number_of_runs), desc=f"Running Simulations for {armor}-{env}", leave=False):
                params = {
                    "blue_stock": blue_stock,
                    "red_stock": red_stock,
                    "direction_deviation": direction_deviation,
                    "armor_type": armor,
                    "environment": env
                }
                result = run_simulation(params, full_log=False) # Set plot=False for multi-run
                # pp.pprint(result)

                # Accumulate metrics
                total_blue_remaining += result['blue']['stock']
                total_effective_movement += result['blue']['patrol_distance'] * blue_stock
                total_blue_kills += result['blue']['kills']
                # Red patrols can respawn, so we need to account for defeated red patrols.
                for red in result['red_patrols']:
                    total_red_remaining += red['stock']
                    total_red_spawned += red_stock
                    total_red_kills += red['kills']

                tbk_list.append(result['red']['kills'])
                thk_list.append(result['blue']['kills'])
                tpd_list.append(int(result['blue']['patrol_distance'] / 1000))

            average_blue_remaining = total_blue_remaining / number_of_runs
            average_red_remaining = total_red_remaining / number_of_runs
            average_effective_movement = total_effective_movement / number_of_runs
            average_blue_lethality = total_blue_kills / blue_stock
            average_red_lethality = total_red_kills / total_red_spawned

            results.append({
                "Armor": armor,
                "Environment": env,
                "Average_Blue_Remaining": average_blue_remaining,
                "Average_red_Remaining": average_red_remaining,
                "Average_Effective_Movement": average_effective_movement,
                "Average_Blue_Lethality": average_blue_lethality,
                "Average_red_Lethality": average_red_lethality

            })
        plot_histograms(TBK, "Blue Kills", "Number of Blue Kills", "Frequency", step=5)
        plot_histograms(THK, "Hostile Kills", "Number of Hostile Kills", "Frequency", step=1)
        plot_histograms(TPD, "Patrol Distance", "Patrol Distance (km)", "Frequency", step=5)

        plot_line(results, armor_types, 'Average_Blue_Remaining', f"Average Blue Remaining vs Environment ({number_of_runs} simulations per combination)", f"Average Blue Remaining (out of {blue_stock})")
        plot_line(results, armor_types, 'Average_red_Remaining', f"Average Hostiles Remaining vs Environment ({number_of_runs} simulations per combination)", f"Average Hostiles Remaining (out of {red_stock})")
        plot_line(results, armor_types, 'Average_Blue_Lethality', f"Average Blue Lethality vs Environment ({number_of_runs} simulations per combination)", "Average Blue Lethality")

        df_results = pd.DataFrame(results)
        display(df_results)

run_button = Button(description=f"Run All Armor/Threat Combinations ({number_of_runs} times each)")
run_button.on_click(run_all_combinations)

VBox([run_button, output])
